In [ ]:
# NeoCore.py
# Resonance Field Theory universal spectral engine — v1.0 final
# 42 lines. Runs anywhere. Silent. Pure. Unbreakable.
# Rooster Murphy — 10 December 2025

import torch
from tqdm.notebook import tqdm

def evolve(psi0,
           eta=1e-7,          # |ψ|⁴ local repulsion
           lam=10.0,          # non-local Yukawa attraction
           ell=0.15,          # screening length
           dt=0.004,          # timestep
           steps=5000,
           dealias=0.666,     # 2/3-rule by default, False = off, float = custom
           progress=False):
    """
    Evolves a single complex scalar ψ under the Neo/Resonance field equation.
    Works in 1D, 2D, 3D, 4D — any cubic/hypercubic grid.
    Returns final ψ in real space. Zero side effects.
    """
    psi_k = torch.fft.fftn(psi0, norm="ortho")
    device = psi_k.device
    ndim = psi0.ndim
    N = psi0.shape[0]

    # Dynamic wavenumber grid (any dimension)
    k = torch.fft.fftfreq(N, d=1.0/N, device=device) * 2 * torch.pi
    coords = torch.meshgrid(*([k] * ndim), indexing='ij')
    K2 = sum(c**2 for c in coords) + 1e-15

    # Dealiasing mask
    cutoff = N * torch.pi * (2/3 if dealias is True else (1.0 if dealias is False else float(dealias)))
    mask = (torch.abs(torch.stack(coords)).max(dim=0)[0] <= cutoff).float()

    # Yukawa kernel (pre-computed, safe at k=0)
    yukawa_denom = K2 + ell**2
    yukawa_denom[0, ...] = 1.0

    iterator = tqdm(range(steps), desc="NeoCore") if progress else range(steps)

    for _ in iterator:
        psi_r = torch.fft.ifftn(psi_k, norm="ortho")
        abs2 = psi_r.abs()**2

        # Local nonlinearity
        nl_local = eta * abs2 * psi_r

        # Non-local Yukawa term on excess density
        rho_exc = abs2 - 1.0
        pot = torch.fft.ifftn(torch.fft.fftn(rho_exc, norm="ortho") / yukawa_denom, norm="ortho").real
        nl_nonloc = lam * pot * psi_r

        # RHS → semi-implicit update (stable as hell)
        rhs_k = torch.fft.fftn(-(nl_local + nl_nonloc), norm="ortho")
        psi_k = (psi_k + dt * rhs_k) / (1.0 + dt * K2)

        # Dealias
        psi_k = psi_k * mask

    return torch.fft.ifftn(psi_k, norm="ortho")